In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import time as t
import pickle
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os
import glob
import json
import re
import unicodedata

## Выгрузка данных по api, формирование датафрейма
Анализ полученных данных в сформированном датафрейме выполняется в другом файле

**1. Подготовка к выгрузке, задаем параметры, создаем списки под данные в колонках**

In [54]:
# Создадим пустые списки для выгрузки данных по колонкам будущей таблицы

add_id, link, title, price, land_cat, square, ads_time, person_type, region, city = [],[],[],[],[],[],[],[],[],[]
address, description, nedvigimost_type, coords, location = [],[],[],[],[]
time_source_created, time_source_updated, is_actual = [],[],[]

In [55]:
# Задаем параметры поиска (см. инструкцию на сайте ads-api.ru)

token = '94c0112f4aade174cdf207fdec13d888'
category_id = '5'
start_date = datetime(2023, 1, 1, 0, 0, 0)
end_date = datetime(2023, 2, 3, 19, 42, 12) 
source = '1'

**2. Выгружаем данные по api, заполняем списки для формирования датафрейма**

In [56]:
'Задаем макс. количество объявлений за 1 запрос (тест - 50, платно - 1000) и длительность паузы между запросами'
max_ads_request = 1000
sleep_time = 6

'Запускаем цикл'
while end_date > start_date and len(add_id) <= 500000:
    'Задаем временной интервал поиска'
    start_for_url = start_date.strftime('%Y-%m-%d+%H:%M:%S')
    end_for_url = end_date.strftime('%Y-%m-%d+%H:%M:%S')
    
    'Составляем url запроса (см. инструкцию на сайте ads-api.ru)'
    url = f'https://ads-api.ru/main/api?user=dima.greensfan@mail.ru&token={token}' +\
          f'&category_id={category_id}' +\
          f'&date1={start_for_url}' +\
          f'&date2={end_for_url}' +\
          f'&nedvigimost_type=1' +\
          f'&source={source}' +\
          f'&withphone=0'
    
    'Отправляем запрос и формируем словарь из json-ответа, цикл нужен для автоповторов запроса'
    bad_response_count = 0    
    while True:
        r = requests.get(url)
        r_info = r.json()
        
        if 'code' in r_info and r_info['code'] == 200:
            print(f'Успешный запрос на временном периоде {start_date} - {end_date}')
            break
        else:
            bad_response_count += 1
            if bad_response_count == 10:
                print(f'{bad_response_count} неудачных запросов на временном периоде',
                     f'{start_date} - {end_date}')
                break
            
    if 'data' in r_info:
        r_info_data = r_info['data']
        
        for el in r_info_data:
            try:
                add_id.append(el['id'])
            except:
                add_id.append('-')
            try:
                link.append(el['url'])
            except:
                link.append('-')
            try:
                title.append(el['title'])
            except:
                title.append('-')
            try:
                price.append(el['price'])
            except:
                price.append(0)
            try:
                land_cat.append(el['param_4313'])
            except:
                land_cat.append('-')
            try:
                square.append(el['param_4616'])
            except:
                square.append('-')
            try:
                ads_time.append(el['time'])
            except:
                ads_time.append('-')
            try:
                person_type.append(el['person_type'])
            except:
                person_type.append('-')
            try:
                region.append(el['region'])
            except:
                region.append('-')
            try:
                city.append(el['city1'])
            except:
                city.append('-')
            try:
                address.append(el['address'])
            except:
                address.append('-')
            try:
                description.append(el['description'])
            except:
                description.append('-')
            try:
                nedvigimost_type.append(el['nedvigimost_type'])
            except:
                nedvigimost_type.append('-')
            try:
                coords.append(f"{el['coords']['lat']}, {el['coords']['lng']}")
            except:
                coords.append('-')
            try:
                location.append(el['param_4443'])
            except:
                location.append('-')
            try:
                time_source_created.append(el['time_source_created'])
            except:
                time_source_created.append('-')
            try:
                time_source_updated.append(el['time_source_updated'])
            except:
                time_source_updated.append('-')
            try:
                is_actual.append(el['is_actual'])
            except:
                is_actual.append('-')
        
        print(f'Выгружены данные по {len(r_info_data)} объявлениям: временной интервал {start_date} - {end_date}')
        
        'Проверка кол-ва объявлений и изменение конечной даты на дату самого раннего объявл. из загруженных'
        if len(r_info_data) == max_ads_request:
            if ads_time[-1] != '-':
                new_end_date = datetime.strptime(ads_time[-1], '%Y-%m-%d %H:%M:%S')
                if new_end_date < end_date:
                    end_date = new_end_date
                else:
                    end_date -= timedelta(hours=3)
            else:
                end_date -= timedelta(hours=3)
        else:
            end_date = start_date
    else:
        print(f'В ответе от сервера нет данных: временной интервал {start_date} - {end_date}')
        end_date -= timedelta(hours=3)
    
    t.sleep(sleep_time)

Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-02-03 19:42:12
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-02-03 19:42:12
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-02-03 17:56:41
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-02-03 17:56:41
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-02-03 16:23:06
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-02-03 16:23:06
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-02-03 14:45:57
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-02-03 14:45:57
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-02-03 13:14:33
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-02-03 13:14:33
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-02-03 11:50:06
Выгружены данные по 1000 объявл

Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-30 08:15:06
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-30 08:15:06
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-30 06:13:13
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-30 06:13:13
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-29 22:04:40
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-29 22:04:40
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-29 20:02:47
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-29 20:02:47
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-29 18:07:20
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-29 18:07:20
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-29 16:25:10
Выгружены данные по 1000 объявл

Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-25 15:34:59
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-25 15:34:59
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-25 14:12:20
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-25 14:12:20
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-25 12:30:25
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-25 12:30:25
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-25 10:55:44
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-25 10:55:44
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-25 09:34:30
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-25 09:34:30
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-25 07:45:15
Выгружены данные по 1000 объявл

Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-21 12:27:16
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-21 12:27:16
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-21 09:55:44
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-21 09:55:44
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-21 07:29:42
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-21 07:29:42
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-21 05:04:34
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-21 05:04:34
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-21 03:42:25
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-21 03:42:25
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-21 02:24:27
Выгружены данные по 1000 объявл

Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-17 08:06:51
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-17 08:06:51
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-17 04:17:39
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-17 04:17:39
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-17 00:15:38
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-17 00:15:38
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-16 23:20:09
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-16 23:20:09
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-16 22:12:42
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-16 22:12:42
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-16 20:53:51
Выгружены данные по 1000 объявл

Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-12 23:07:45
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-12 23:07:45
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-12 21:51:58
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-12 21:51:58
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-12 20:16:57
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-12 20:16:57
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-12 17:51:23
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-12 17:51:23
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-12 14:43:19
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-12 14:43:19
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-12 11:30:50
Выгружены данные по 1000 объявл

Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-08 17:26:43
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-08 17:26:43
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-08 15:52:14
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-08 15:52:14
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-08 14:27:03
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-08 14:27:03
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-08 12:29:07
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-08 12:29:07
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-08 10:57:01
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-08 10:57:01
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-08 09:07:11
Выгружены данные по 1000 объявл

Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-03 21:32:07
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-03 21:32:07
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-03 19:26:31
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-03 19:26:31
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-03 17:35:45
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-03 17:35:45
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-03 15:37:26
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-03 15:37:26
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-03 13:51:49
Выгружены данные по 1000 объявлениям: временной интервал 2023-01-01 00:00:00 - 2023-01-03 13:51:49
Успешный запрос на временном периоде 2023-01-01 00:00:00 - 2023-01-03 12:09:49
Выгружены данные по 1000 объявл

**3. Проверка полученных списков**

In [57]:
print(f'id: {len(add_id)} записей')
print(f'Ссылки: {len(link)} записей')
print(f'Название: {len(title)} записей')
print(f'Цена: {len(price)} записей')
print(f'Категория ЗУ: {len(land_cat)} записей')
print(f'Площадь: {len(square)} записей')
print(f'Дата объявления: {len(ads_time)} записей')
print(f'Продавец: {len(person_type)} записей')
print(f'Регион: {len(region)} записей')
print(f'Город: {len(city)} записей')
print(f'Адрес: {len(address)} записей')
print(f'Описание: {len(description)} записей')
print(f'Тип объявления: {len(nedvigimost_type)} записей')
print(f'Координаты: {len(coords)} записей')
print(f'Местоположение: {len(location)} записей')
print(f'Дата создания объявления: {len(time_source_created)} записей')
print(f'Дата обновления объявления: {len(time_source_updated)} записей')
print(f'Актуальность объявления: {len(is_actual)} записей')

id: 349695 записей
Ссылки: 349695 записей
Название: 349695 записей
Цена: 349695 записей
Категория ЗУ: 349695 записей
Площадь: 349695 записей
Дата объявления: 349695 записей
Продавец: 349695 записей
Регион: 349695 записей
Город: 349695 записей
Адрес: 349695 записей
Описание: 349695 записей
Тип объявления: 349695 записей
Координаты: 349695 записей
Местоположение: 349695 записей
Дата создания объявления: 349695 записей
Дата обновления объявления: 349695 записей
Актуальность объявления: 349695 записей


In [23]:
'Уравняем число элементов в списках'
# add_id.pop()
# link.pop()
# title.pop()
# price.pop()
# land_cat.pop()

'Поселений (ИЖС)'

**4. Создание датафрейма из списков**

In [25]:
# dop_df = pd.DataFrame({'add_id': add_id,
#                    'link': link,
#                    'title': title,
#                    'price': price,
#                    'land_cat': land_cat,
#                    'square': square,
#                    'ads_time': ads_time,
#                    'person_type': person_type,
#                    'region': region,
#                    'city': city,
#                    'address': address,
#                    'description': description,
#                    'nedvigimost_type': nedvigimost_type,
#                    'coords': coords,
#                    'location': location,
#                    'time_source_created': time_source_created,
#                    'time_source_updated': time_source_updated,
#                    'is_actual': is_actual})


KeyboardInterrupt



In [ ]:
df.shape

In [58]:
'1. Если загружаем лоты в df впервые'

df = pd.DataFrame({'add_id': add_id,
                   'link': link,
                   'title': title,
                   'price': price,
                   'land_cat': land_cat,
                   'square': square,
                   'ads_time': ads_time,
                   'person_type': person_type,
                   'region': region,
                   'city': city,
                   'address': address,
                   'description': description,
                   'nedvigimost_type': nedvigimost_type,
                   'coords': coords,
                   'location': location,
                   'time_source_created': time_source_created,
                   'time_source_updated': time_source_updated,
                   'is_actual': is_actual})

In [59]:
'Сохранение по частям из-за большого кол-ва строк'

# df.to_csv('all_ads_landplots_3.csv')

In [60]:
'Сериализация в формат pickle по частям'

# with open('all_ads_landplots_3.pkl', 'wb') as file:
#     pickle.dump(df, file)

# ЗАКОНЧИЛ ЗДЕСЬ 02.04

### ЗАГРУЗКА ДАННЫХ ПО ЧАСТЯМ

*4.1. Удаляем полные дубликаты строк*

In [110]:
# df = df.drop_duplicates()

*4.2. Проверка на пустые ячейки*

In [122]:
df.head(1)

,add_id,link,title,price,land_cat,square,ads_time,person_type,region,city,address,description,nedvigimost_type,coords,location,time_source_created,time_source_updated,is_actual
0,619821774,https://www.avito.ru/krasnodar/zemelnye_uchast...,Участок 7 сот. (промназначения),4200000,Промназначения,7.0,2024-02-29 17:45:39,Частное лицо,Краснодарский край,Краснодар,"Республика Адыгея, Тахтамукайский р-н, Яблонов...","Продам участок в шаговой доступности школа , м...",Продам,"45.007189, 38.982535",-,2023-10-30 23:09:34,2024-02-29 18:11:52,11


In [145]:
'Проверка всех колонок на отсутствующие значения'
df.isna().sum()

add_id                 0
link                   0
title                  0
price                  0
land_cat               0
square                 0
ads_time               0
person_type            0
region                 0
city                   0
address                0
description            0
nedvigimost_type       0
coords                 0
location               0
time_source_created    0
time_source_updated    0
is_actual              0
dtype: int64

In [146]:
'Локальная проверка колонок (при необходимости)'
df[df.is_actual.isnull()]

In [ ]:
'Заполнение отсутствующих значений (при необходимости)'


*4.2. Проверка возможности форматирования численных данных в колонках*

In [147]:
df.dtypes

add_id                          int64
link                           object
title                          object
price                           int64
land_cat                       object
square                        float64
ads_time               datetime64[ns]
person_type                    object
region                         object
city                           object
address                        object
description                    object
nedvigimost_type               object
coords                         object
location                       object
time_source_created    datetime64[ns]
time_source_updated    datetime64[ns]
is_actual                       int64
dtype: object

In [ ]:
'К типу данных float'
# df.price = df.price.astype('float')

In [ ]:
'Временные колонки к типу данных datetime'

df.ads_time = pd.to_datetime(df.ads_time)
df.time_source_created = pd.to_datetime(df.time_source_created)
df.time_source_updated = pd.to_datetime(df.time_source_updated)

**5. Проверка территориальной принадлежности объявлений к муниципальным районам по координатам**

**Есть геополигоны со списками координат границ всех районов всех регионов РФ в папке "geojson".
  Нужно проверить, внутри какого полигона находятся координаты объявлений и вписать районы в новую колонку**

5.1. Подготовка

In [15]:
'Десериализуем последнюю версию df из pickle файла (при необходимости)'

# pkl_data_path = '/Users/dmitrijdolgopolov/Documents/1_study_python/projects/9_torgi.gov_api' +\
#                 '/new_parser_avito/1_parser/ads_landplots.pkl'

# with open(pkl_data_path, 'rb') as file:
#     df = pickle.load(file)

In [147]:
'Соберем в словарь пути ко всем geojson файлам с полигонами координат муниципальных районов'

current_file = os.path.realpath('__file__')
current_directory = os.path.dirname(current_file)

geojson_files_path = glob.glob(f'{current_directory}/geojson/all_regions/*.geojson')
region_names = [x.split('/')[-1].split('.')[0] for x in geojson_files_path]

regions_geojson_dict = {unicodedata.normalize('NFC', k): v for k, v in zip(region_names, geojson_files_path)}
# print(*[f'{k}: {regions_geojson_dict[k]}' for k in regions_geojson_dict], sep='\n')

In [30]:
'Добавление колонки с районами'

df['district'] = ''

In [152]:
# for el in regions_geojson_dict:
#     print(el, regions_geojson_dict[el])
# for el in df[df.region != ''].region.unique():
#     if any([x.__contains__(el) for x in regions_geojson_dict]):
#         print(f'{el}: Да')
#     else:
#         print(f'{el}: Нет')

5.2. Поиск вхождений координат объявлений в геополигоны по заданным регионам в колонке regions

In [392]:
'''Итерируемся по индексам df и проверяем координаты объявления на нахождение внутри 
   полигонов координат районов известного региона'''

count = 0

print("Объявлений без указания района ", df.query("region != '' & district == ''").shape[0])
for i in df.query("region != '' & district == ''").index:
# for i in df.query("region == 'Московская область' & district == ''").index:

    
    x_coord = df.loc[i, 'coords'].split(',')[0]
    y_coord = df.loc[i, 'coords'].split(',')[-1].strip()
    point = Point(y_coord, x_coord)
    

    try:
        with open(regions_geojson_dict[df.loc[i, 'region']]) as f:
            geodata = json.load(f)
        
        is_found = False
        for el in geodata['features']:
            if is_found:
                break
            for polyg in el['geometry']['coordinates']:
                if is_found:
                    break
                if not isinstance(polyg[0][0], list):
                    polyg_tuple = tuple(tuple(x) for x in polyg)
                    polygon = Polygon(polyg_tuple)
                    if polygon.contains(point):
                        df.loc[i, 'district'] = el['properties']['district']
                        print(f"{df.loc[i, 'region']}: {df.loc[i, 'coords']} - {el['properties']['district']}")
                        is_found = True
                        count += 1
                        break
#                     else:
#                         print(f"{df.loc[i, 'region']}: {df.loc[i, 'coords']}",
#                                   f" в {el['properties']['district']} не найдено")
#                         print(df.loc[i, 'link'])
                elif isinstance(polyg[0][0], list):
                    for polyg_part in polyg:
                        polyg_tuple = tuple(tuple(x) for x in polyg_part)
                        polygon = Polygon(polyg_tuple)
                        if polygon.contains(point):
                            df.loc[i, 'district'] = el['properties']['district']
                            print(f"{df.loc[i, 'region']}: {df.loc[i, 'coords']} - {el['properties']['district']}")
                            is_found = True
                            count += 1
                            break
#                         else:
#                             print(f"{df.loc[i, 'region']}: {df.loc[i, 'coords']}",
#                                   f" в {el['properties']['district']} не найдено")
#                             print(df.loc[i, 'link'])

    except:
        print(f"Ошибка: {df.loc[i, 'link']}")
        raise

print(f'Добавлен район в {count} объявлениях')

Объявлений без указания района  729
Московская область: 55.791035, 37.367086 - Городской Округ Красногорск
Московская область: 56.051451, 37.151158 - Городской Округ Солнечногорск
Московская область: 55.557103, 37.625621 - Ленинский Городской Округ
Московская область: 55.731855, 37.334693 - Одинцовский Городской Округ
Московская область: 55.96997, 37.266457 - Городской Округ Солнечногорск
Московская область: 55.676478, 37.351375 - Одинцовский Городской Округ
Московская область: 55.949108, 37.181413 - Городской Округ Солнечногорск
Московская область: 55.563029, 37.643543 - Ленинский Городской Округ
Московская область: 55.557948, 37.626573 - Ленинский Городской Округ
Московская область: 55.923115, 37.648465 - Городской Округ Мытищи
Московская область: 55.5636420721763, 37.6034127698669 - Ленинский Городской Округ
Московская область: 55.526133, 37.599489 - Ленинский Городской Округ
Московская область: 56.036507, 37.133398 - Городской Округ Солнечногорск
Московская область: 55.733214, 37.3

5.3. Проверка полноты заполнения колонки district и ручная корректировка результатов (при необходимости возвращаемся к п. 5.2)

In [302]:
# df.query("region != '' & district == ''").region.value_counts()

In [382]:
'''Обнаружились неверно указанные регионы для объявлений. Проверим колонку region на соответствие адресу, 
   указанному в колонке address. Если они отличаются, переприсвоим регион по колонке address, соотнеся его
   с ключами в regions_geojson_dict. После переприсваивания региона повторно проверим координаты в новом регионе'''

count = 0
for i in df.query("district == ''").index:
    add_region = df.loc[i, 'region']
    address_region = df.loc[i, 'address'].split(',')[0].strip()
    
    if re.search('[Рр]еспублика', address_region) and not address_region == 'Республика Алтай':
        address_region = re.sub('[Рр]еспублика', '', address_region).strip()
        
        if add_region != address_region:
            for key in regions_geojson_dict:
                if address_region.endswith('ская'):
                    address_region = address_region.replace('ская', '')
                if re.search(address_region, key) and df.loc[i, 'region'] != key:
                    count += 1
                    df.loc[i, 'region'] = key
                    df.loc[i, 'city'] = ''
                    print(key, address_region)
    
    elif re.search('[Оо]бл.', address_region) or re.search('[Кк]рай', address_region):
        address_region = address_region.split()[0]
        if re.search('[Оо]бласть', add_region) or re.search('[Кк]рай', add_region):
            add_region = add_region.split()[0]
        
        if add_region != address_region:
            for key in regions_geojson_dict:
                if re.search(address_region, key) and df.loc[i, 'region'] != key:
                    count += 1
                    df.loc[i, 'region'] = key
                    df.loc[i, 'city'] = ''
                    print(key, address_region)
    
    elif address_region in ('Москва', 'Санкт-Петербург', 'Севастополь') and add_region != address_region:
        for key in regions_geojson_dict:
            if re.search(address_region, key) and df.loc[i, 'region'] != key:
                count += 1
                df.loc[i, 'region'] = key
                df.loc[i, 'city'] = key
                print(key, address_region)
    elif add_region == 'Санкт-Петербург' and re.search('поселение', address_region):
        count += 1
        df.loc[i, 'region'] = 'Ленинградская область'
        df.loc[i, 'city'] = address_region
    elif add_region == 'Крым' and df.loc[i, 'city'] == 'Севастополь':
        count += 1
        df.loc[i, 'region'] = 'Севастополь'
    elif add_region == 'Москва' and address_region != add_region:
        count += 1
        df.loc[i, 'region'] = 'Московская область'
        if df.loc[i, 'city'] == 'Москва':
            df.loc[i, 'city'] = ''
        
    
print(f'Изменен регион в {count} объявлениях')

Изменен регион в 0 объявлениях


In [401]:
# df.query("region == 'Санкт-Петербург' & district == ''").shape
# df.loc[46323, 'address'] = '''г. Севастополь, бух. Казачья, 51/33'''
# df.loc[46323, 'coords'] = '''44.573103, 33.415673'''
# df.loc[46323, 'region'] = '''Севастополь'''
# df.query("region != '' & district == ''").region.value_counts()
df.query("region == 'Московская область' & district == ''").head(2)


,add_id,link,title,price,land_cat,square,ads_time,person_type,region,city,address,description,nedvigimost_type,coords,location,time_source_created,time_source_updated,is_actual,district
4354,631257307,https://www.avito.ru/lms/zemelnye_uchastki/uch...,Участок 20 сот. (личное подсобное хозяйство (л...,6200000,-,20.0,2024-02-29 10:38:52,Агентство,Московская область,ЛМС,река Черничка,Ровный участок с живописным видом вблизи город...,Продам,"55.216356, 37.079904",-,2024-01-30 10:35:34,2024-02-29 11:05:31,11,
7379,629158076,https://www.avito.ru/kokoshkino/zemelnye_uchas...,Участок 6 сот. (ИЖС),6000000,Поселений (ИЖС),6.0,2024-02-28 23:40:01,Агентство,Московская область,Кокошкино,"поселение Марушкинское, квартал № 92",ОТЛИЧНОЕ ПРЕДЛОЖЕНИЕ!\n\nОТ СОБСТВЕННИКА! СТАР...,Продам,"55.589336, 37.142247",-,2023-12-30 23:06:56,2024-02-29 00:06:34,11,


In [402]:
# print(*df.query("region == 'Московская область' & district == ''").link, sep='\n')

In [252]:
# df.query("region != '' & district == ''")[['add_id', 'region', 'city', 'address']]

5.4. Итоговая проверка всех пустых значений в колонке districts во всех подряд геополигонах регионов

In [403]:
'''Итерируемся по индексам df и проверяем координаты объявления на нахождение внутри 
   полигонов координат всех районов из списка файлов с геополигонами'''

count = 0

print("Объявлений без указания района ", df.query("region != '' & district == ''").shape[0])
for i in df.query("region != '' & district == ''").index:
    
    x_coord = df.loc[i, 'coords'].split(',')[0]
    y_coord = df.loc[i, 'coords'].split(',')[-1].strip()
    point = Point(y_coord, x_coord)
    

    try:
        is_found = False
        for reg in regions_geojson_dict:
            if is_found:
                break
                
            with open(regions_geojson_dict[reg]) as f:
                geodata = json.load(f)

            for el in geodata['features']:
                if is_found:
                    break
                for polyg in el['geometry']['coordinates']:
                    if is_found:
                        break
                    if not isinstance(polyg[0][0], list):
                        polyg_tuple = tuple(tuple(x) for x in polyg)
                        polygon = Polygon(polyg_tuple)
                        if polygon.contains(point):
                            df.loc[i, 'district'] = el['properties']['district']
                            print(f"Старый регион {df.loc[i, 'region']}")
                            
                            df.loc[i, 'region'] = reg
                            print(f"Новый регион {df.loc[i, 'region']}: {df.loc[i, 'coords']} - ", 
                                  f"{el['properties']['district']}")
                            is_found = True
                            count += 1
                            break

                    elif isinstance(polyg[0][0], list):
                        for polyg_part in polyg:
                            polyg_tuple = tuple(tuple(x) for x in polyg_part)
                            polygon = Polygon(polyg_tuple)
                            if polygon.contains(point):
                                df.loc[i, 'district'] = el['properties']['district']
                                print(f"Старый регион {df.loc[i, 'region']}")
                                
                                df.loc[i, 'region'] = reg
                                print(f"Новый регион {df.loc[i, 'region']}: {df.loc[i, 'coords']} - ", 
                                      f"{el['properties']['district']}")
                                is_found = True
                                count += 1
                                break

    except:
        print(f"Ошибка: {df.loc[i, 'link']}")
        raise

print(f'Добавлен район в {count} объявлениях')

Объявлений без указания района  686
Старый регион Крым
Новый регион Севастополь: 44.480393, 33.796229 -  Орлиновский муниципальный округ
Старый регион Крым
Новый регион Севастополь: 44.437255, 33.778748 -  Орлиновский муниципальный округ
Старый регион Крым
Новый регион Севастополь: 44.609648, 33.585861 -  город Инкерман
Старый регион Московская область
Новый регион Москва: 55.216356, 37.079904 -  Троицкий административный округ
Старый регион Карелия
Новый регион Ленинградская область: 60.842669, 31.460636 -  Волховский район
Старый регион Свердловская область
Новый регион Челябинская область: 56.104953, 61.548943 -  Каслинский район
Старый регион Алтайский край
Новый регион Республика Алтай: 51.31531496123422, 85.56260950952608 -  Шебалинский район
Старый регион Московская область
Новый регион Москва: 55.589336, 37.142247 -  Новомосковский административный округ
Старый регион Московская область
Новый регион Москва: 55.589336, 37.142247 -  Новомосковский административный округ
Старый ре

Старый регион Московская область
Новый регион Москва: 55.453723, 36.989812 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.472464, 36.901417 -  Троицкий административный округ
Старый регион Курская область
Новый регион Воронежская область: 51.6814869975659, 38.3225803553772 -  Нижнедевицкий район
Старый регион Московская область
Новый регион Москва: 55.472464, 36.901417 -  Троицкий административный округ
Старый регион Костромская область
Новый регион Ярославская область: 57.780344, 40.534618 -  Некрасовский район
Старый регион Московская область
Новый регион Москва: 55.362542, 37.386005 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.476597, 37.41281 -  Новомосковский административный округ
Старый регион Московская область
Новый регион Москва: 55.367157, 37.230201 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.598019, 37.102694 -  Новомосковский администр

Старый регион Московская область
Новый регион Москва: 55.402674, 37.380875 -  Троицкий административный округ
Старый регион Ставропольский край
Новый регион Карачаево-Черкесия: 43.700856, 42.662888 -  Малокарачаевский район
Старый регион Московская область
Новый регион Москва: 55.425175, 37.373886 -  Троицкий административный округ
Старый регион Московская область
Новый регион Тульская область: 54.622596, 38.358081 -  Веневский район
Старый регион Московская область
Новый регион Владимирская область: 56.521701, 38.45332 -  Александровский район
Старый регион Московская область
Новый регион Москва: 55.449562, 37.218765 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.409058, 37.167516 -  Троицкий административный округ
Старый регион Краснодарский край
Новый регион Адыгея: 44.997451, 38.998768 -  Тахтамукайский район
Старый регион Московская область
Новый регион Москва: 55.406186, 37.124577 -  Троицкий административный округ
Старый регион Москов

Старый регион Московская область
Новый регион Москва: 55.506219, 37.437514 -  Новомосковский административный округ
Старый регион Московская область
Новый регион Москва: 55.212889, 37.060545 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.573622, 37.108434 -  Новомосковский административный округ
Старый регион Московская область
Новый регион Москва: 55.581597, 37.139749 -  Новомосковский административный округ
Старый регион Московская область
Новый регион Москва: 55.341669, 37.177191 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.423688, 37.283157 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.423642, 37.376159 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.584207, 37.483912 -  Новомосковский административный округ
Старый регион Московская область
Новый регион Москва: 55.40089, 36.912601 -  Троицкий администра

Старый регион Крым
Новый регион Севастополь: 44.448845, 33.784731 -  Орлиновский муниципальный округ
Старый регион Московская область
Новый регион Тульская область: 54.768443, 37.996805 -  Ясногорский район
Старый регион Челябинская область
Новый регион Башкортостан: 53.642052, 58.802632 -  Абзелиловский район
Старый регион Ленинградская область
Новый регион Псковская область: 58.606277, 28.836361 -  Плюсский район
Старый регион Московская область
Новый регион Владимирская область: 56.121431, 38.636019 -  Киржачский район
Старый регион Московская область
Новый регион Рязанская область: 54.910764651013736, 39.51058589020212 -  Рыбновский район
Старый регион Крым
Новый регион Севастополь: 44.506608, 33.824867 -  Орлиновский муниципальный округ
Старый регион Крым
Новый регион Севастополь: 44.464531, 33.848376 -  Орлиновский муниципальный округ
Старый регион Республика Алтай
Новый регион Алтайский край: 51.947092, 85.810085 -  Алтайский район
Старый регион Крым
Новый регион Севастополь: 44

Старый регион Краснодарский край
Новый регион Адыгея: 45.047800391086426, 38.89358993330568 -  Тахтамукайский район
Старый регион Московская область
Новый регион Москва: 55.477245, 37.388367 -  Новомосковский административный округ
Старый регион Московская область
Новый регион Москва: 55.423642, 37.376159 -  Троицкий административный округ
Старый регион Хабаровский край
Новый регион Еврейская автономная область: 48.538557, 134.955405 -  Смидовичский район
Старый регион Калужская область
Новый регион Тульская область: 54.691178, 37.25821 -  Заокский район
Старый регион Московская область
Новый регион Владимирская область: 56.459999, 38.433593 -  Александровский район
Старый регион Московская область
Новый регион Москва: 55.39383, 37.37748 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.490182, 37.543973 -  Новомосковский административный округ
Старый регион Московская область
Новый регион Москва: 55.220033, 37.059171 -  Троицкий административн

Старый регион Московская область
Новый регион Тверская область: 56.520454, 35.650217 -  Калининский район
Старый регион Московская область
Новый регион Тверская область: 56.520454, 35.650217 -  Калининский район
Старый регион Московская область
Новый регион Тверская область: 56.520454, 35.650217 -  Калининский район
Старый регион Московская область
Новый регион Тверская область: 56.520454, 35.650217 -  Калининский район
Старый регион Московская область
Новый регион Тверская область: 56.520454, 35.650217 -  Калининский район
Старый регион Московская область
Новый регион Тверская область: 56.520454, 35.650217 -  Калининский район
Старый регион Московская область
Новый регион Тверская область: 56.520454, 35.650217 -  Калининский район
Старый регион Московская область
Новый регион Тверская область: 56.520454, 35.650217 -  Калининский район
Старый регион Московская область
Новый регион Тверская область: 56.520454, 35.650217 -  Калининский район
Старый регион Московская область
Новый регион 

Старый регион Московская область
Новый регион Москва: 55.642738, 37.27111 -  Новомосковский административный округ
Старый регион Новгородская область
Новый регион Ленинградская область: 59.21999, 31.483552 -  Тосненский район
Старый регион Московская область
Новый регион Тульская область: 54.752563, 37.937878 -  Ясногорский район
Старый регион Московская область
Новый регион Москва: 55.428996, 37.382268 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.487433, 37.253692 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.213022, 37.0605 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.325411, 37.100808 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.576131, 37.126598 -  Новомосковский административный округ
Старый регион Московская область
Новый регион Москва: 55.214543, 37.080326 -  Троицкий административный округ
Стар

Старый регион Московская область
Новый регион Москва: 55.552637, 37.195014 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.597398, 37.141645 -  Новомосковский административный округ
Старый регион Московская область
Новый регион Москва: 55.427974, 37.385789 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.443552, 37.32783 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.51431, 37.123068 -  Троицкий административный округ
Старый регион Московская область
Новый регион Калужская область: 54.8272415925529, 37.2082280474723 -  Тарусский район
Старый регион Московская область
Новый регион Москва: 55.477342, 37.388179 -  Новомосковский административный округ
Старый регион Московская область
Новый регион Москва: 55.514075, 37.122475 -  Троицкий административный округ
Старый регион Московская область
Новый регион Москва: 55.435764, 37.328459 -  Троицкий административн

In [419]:
df = df.query("district != ''")

**5. Сохранение датафрейма в csv файл для дальнейшего анализа**

In [44]:
# df.to_csv('all_ads_landplots.csv')

In [45]:
'Сериализация в формат pickle (сохраняет все форматы данных в колонках при загрузке в другом блокноте!!)'

# with open('all_ads_landplots.pkl', 'wb') as file:
#     pickle.dump(df, file)